In [31]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import plotly.express as px


filenames = ['freq_resp_45-55hz_5.5V.csv', 'freq_resp_55-45hz_5.5V.csv']

file_path = 'data/'

In [35]:
min_displacement_mm = 0
max_displacement_mm = 1

max_displ_x = 0.0061
min_displ_x = 0.0235

max_displ_y = 0.0056
min_displ_y = 0.0331

def conv_displacement_to_mm_x(x):
    return (x - min_displ_x) * (max_displacement_mm - min_displacement_mm) / \
           (max_displ_x - min_displ_x) + min_displacement_mm

def conv_displacement_to_mm_y(x):
    return (x - min_displ_y) * (max_displacement_mm - min_displacement_mm) / \
           (max_displ_y - min_displ_y) + min_displacement_mm

def modify_waves(waves_data):
    neg = False
    print('mopslik')
    for i in np.arange(len(waves_data)):
        if i == 0 or (abs(waves_data[i]) < 0.05 and abs(waves_data[i-1]) > 0.05):
            neg = not neg

        if neg:
            waves_data[i] = -waves_data[i]

    return waves_data


In [33]:
data = []

# Read data files.
for filename in filenames:
    file = pd.read_csv(file_path + filename)
    data.append(file)

In [34]:
print(max(data[0]['x_val']))
print(min(data[0]['x_val']))

print(max(data[0]['y_val']))
print(min(data[0]['y_val']))

0.0278103616813294
0.0060117302052785
0.0330889540566959
0.0055718475073313


In [36]:
for df in data:
    with Pool(cpu_count()) as p:
        if 'x_val' in df.columns:
            df['x_val'] = p.map(conv_displacement_to_mm_x, df['x_val'])
        df['y_val'] = p.map(conv_displacement_to_mm_y, df['y_val'])
#         if 'x_val' in df.columns:
#             df['x_val'], df['y_val'] = p.map(modify_waves, (df['x_val'], df['y_val']))
#         else:
#             df['y_val'] = modify_waves(df['y_val'])


In [50]:
df['x_val'] = df['x_val'] + 0.1

In [37]:
for i, df in enumerate(data):
    df.to_csv(file_path + filenames[i].split('.csv')[0] + '_processed.csv')